# Steam reviews deep dive

This notebook will act as a live document as I use it to source, clean and analyze a dataset sourced from Steam. The notebook will overall be split into two sections:
* Getting the database
* Analyzing the database

Once the database is formed, it will most likely be huge. Instead of storing it on github, I will upload it to kaggle, at which point I will load said database for the sections afterwards. 



## Steam Reviews Get!


#### Objectives
* Source a dataset of all English reviews on steam!
    * Access publicly available api calls for store.steampowered
    * end with a dataframe for which a row will hold the app title/id, review author details, and review details
* Write and save dataset to Kaggle for convenience
* Thoroughly document process to help learn and also later reference as a tutorial

#### Concerns
* Avoiding DOS
    * I'm making an obscene number of requests from Valve what happens if I request too much?
    * Can I limit the amount of requests I make and request in a piecemeal fashion?
        * From research, it looks like steam limits the amount of requests allowed to 100,000 per calendar day. 
        * Approach two different ways: sleep an amount every request to ensure I don't go over (865ms per request) or count requests via a global variable and sleep on 100,000
        

In [2]:
import pandas as pd
import numpy as np
import requests
import time

In [51]:


URL= "https://store.steampowered.com/appreviews/317710?json=1"
cursor = '*'
PARAMS = {
    'filter':'updated',
    'language':'english',
    'cursor': cursor,
    'purchase_type':'all',
    'num_per_page' : 1,
}

1533420
r = requests.get(URL, PARAMS)
r=r.json()

In [52]:
print(r)

print(r["query_summary"]['total_reviews'])
print(r["cursor"])

{'success': 1, 'query_summary': {'num_reviews': 1, 'review_score': 8, 'review_score_desc': 'Very Positive', 'total_positive': 346, 'total_negative': 50, 'total_reviews': 396}, 'reviews': [{'recommendationid': '115975941', 'author': {'steamid': '76561198091142053', 'num_games_owned': 239, 'num_reviews': 2, 'playtime_forever': 6438, 'playtime_last_two_weeks': 9, 'playtime_at_review': 6428, 'last_played': 1655940748}, 'language': 'english', 'review': "If you like anything you see of this, or you go and play the Demo version and like what you see, the whole game is exactly as shown. Fast paced, quick to get started (as in you die and can, within a second, start again), and all about memorizing the different segments of each level. Not a forgiving game, and if you have anger issues, be prepared to step back and breath. But really rewarding when you finally Beat it and get to level 9 and not just survive, but flourish.\n\nWarning, How the game plays it directly related to your frame rate. Ev

In [94]:
# print(r["reviews"])
df = pd.json_normalize(r["reviews"])
# df.head()

In [20]:
def getAppReviews(app, step, pause): #get all reviews associated with an app given an appid, size to get reviews with, and a pause between each request (seconds)
    URL = "https://store.steampowered.com/appreviews/" + str(app) +"?json=1"

    tally=step
    PARAMS = {
        'language':'english',
        'filter':'updated',
        'purchase_type':'all',
        'cursor': '*',  #initial value defined in steam's documentation
        'num_per_page' : step,
    }

    r = requests.get(URL, PARAMS)
    r=r.json()
    nCursor = {'cursor' : r["cursor"]}
    PARAMS.update(nCursor)
    total = r["query_summary"]['total_reviews']
    df = pd.json_normalize(r["reviews"])
    while tally < total:
        time.sleep(pause)
        r = requests.get(URL, PARAMS)
        r=r.json()
        df = pd.concat([df, (pd.json_normalize(r["reviews"]))])
        tally += step
        nCursor = {'cursor' : r["cursor"]}
        PARAMS.update(nCursor)
    return df
    

In [70]:
df = getAppReviews(317710, 51, 0.87)

In [65]:
df.shape

(396, 20)

We now have a method to get reviews given a specific app. Next we get a list of all apps to call from.


In [18]:
apps = requests.get("https://api.steampowered.com/ISteamApps/GetAppList/v2/")
apps=apps.json()
df = pd.json_normalize(apps["applist"]["apps"])
df.head()
dc=df.replace(r'^\s*$', np.nan, regex=True)
dc=dc.dropna()
dc.shape
dc.reset_index(drop=True, inplace=True)
dc.head()
apps = dc.sort_values(by=['appid']).reset_index()
##write apps to csv file


In [27]:
last = pd.read_csv('last.csv')
last = last.iloc[0]['appid']
#read apps to csv file
current = pd.DataFrame()
for id in apps[apps['appid'] > last]['appid']:
    current = getAppReviews(id, 51, 0.87)
    if not current.empty:
        name=(df.loc[df['appid'] == id,'name'].values[0]) #get current name
        current.insert(0,'name',name) #add column of name to df for labeling
        current.insert(0,'appid',id) #add column of appid to df
        current.to_csv('reviews.csv', mode='a', index=False, header=False)
        current.tail(1).to_csv('last.csv')
        #all_reviews=pd.concat([all_reviews, current], ignore_index=True) #append to larger db
all_reviews.to_csv("reviews.csv")
    


(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)


KeyboardInterrupt: 

What am I doing next? 

* write code to iterate through the applist
    * check if pd is empty
    * add to major overall database
* save database to file to upload to kaggle

New idea:
* we know that the database will be huge and will take days to compile, so why don't we write a process that can start and stop regardless of user intervention? I still want to turn my computer off at night. 
    * instead of appending to a total database, we write each one to add to an existing csv file. 
    * When starting the process up again, we check the csv to know where we should start. this way we can start and stop without having to worry about tracking where we stopped last.
    

In [42]:
#Test Value cell
apptest = 1462040
df = getAppReviews(apptest, 51, 0.87)
print(df.empty)


False


In [43]:
id=1462040
df.insert(0,dc,value)

,recommendationid,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,118403162,english,"Remake and it's on PC, what more can you ask f...",1657218023,1657218023,True,0,0,0,0,True,False,False,76561198017661797,46,3,870,870,870,1657217906
1,118402906,english,Great revisit to the game. I loved the cut sce...,1657217747,1657217747,True,0,0,0,0,True,False,False,76561198073669555,41,2,3096,3096,3071,1657218885
2,118402709,english,"This is game isn't just bad, I can almost feel...",1657217543,1657217543,False,0,0,0.458998531103134155,0,True,False,False,76561198069565559,176,11,4557,4364,4557,1657209829
3,117643060,english,i love Aerith,1656238808,1657213036,True,0,0,0,0,True,False,False,76561198860089878,135,6,1343,1343,1343,1657117141
4,118397635,english,this remake is in good hands,1657211954,1657211954,True,0,0,0,0,True,False,False,76561198050524284,61,21,2970,2970,2883,1657217271


In [55]:
dc.index[dc['appid']=='1462040']


Int64Index([], dtype='int64')

In [33]:
d = {'sample_col1': [1, 2, 123],
     'sample_col2': [4, 5, 1], 
     'sample_col3': [7, 8, 97]} 

df = pd.DataFrame(d) 
tdf = pd.DataFrame()
print(df.loc[df['sample_col1'] == 123])
print(df.loc[df['sample_col1'] == 123,'sample_col3'].values[0])
df.tail(1).to_csv('last.csv')

   sample_col1  sample_col2  sample_col3
2          123            1           97
97


In [48]:
n = df.sort_values(by=['sample_col2']).reset_index()
print(n)
print(n[n['sample_col2'] > 4].iloc[0]['sample_col2'])
for row in n[n['sample_col2'] > 1]:
    print(row)


   index  sample_col1  sample_col2  sample_col3
0      2          123            1           97
1      0            1            4            7
2      1            2            5            8
5
index
sample_col1
sample_col2
sample_col3
